In [12]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os

import pandas as pd
import scipy.stats as stats
import sklearn.model_selection as model_selection
# from sklearn.random_projection import Pylance 
from sklearn.metrics import precision_score
import sklearn.ensemble as ensemble

random_seed = 42

In [13]:
# Load data (already fixed path)
df_stock = pd.read_csv(r"../data/infolimpioavanzadoTarget.csv")
df_stock.head()

,date,open,high,low,close,adjclose,volume,ticker,RSIadjclose15,RSIvolume15,...,high-15,K-15,D-15,stochastic-k-15,stochastic-d-15,stochastic-kd-15,volumenrelativo,diff,INCREMENTO,TARGET
0,2022-01-03,17.799999,18.219000,17.500000,17.760000,17.760000,106600,ASLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.919758,-1.900001,-9.664295,0
1,2022-01-04,17.700001,18.309999,17.620001,17.660000,17.660000,128700,ASLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.110440,-1.379999,-7.247895,0
2,2022-01-05,17.580000,17.799999,16.910000,16.950001,16.950001,103100,ASLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.889560,-0.930000,-5.201344,0
3,2022-01-06,16.650000,16.879999,16.139999,16.170000,16.170000,173600,ASLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.497843,-0.360000,-2.177856,0
4,2022-01-07,16.219999,16.290001,15.630000,15.710000,15.710000,137800,ASLE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.188956,-0.120000,-0.758054,0


In [14]:
df_stock.info()
df_stock.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7781 entries, 0 to 7780
Columns: 1285 entries, date to TARGET
dtypes: float64(1280), int64(3), object(2)
memory usage: 76.3+ MB


c:\Users\HomePC\Documents\PROJECTS\stock-market-predictor\backend\venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\HomePC\Documents\PROJECTS\stock-market-predictor\backend\venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\HomePC\Documents\PROJECTS\stock-market-predictor\backend\venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\HomePC\Documents\PROJECTS\stock-market-predictor\backend\venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\HomePC\Documents\PROJECTS\stock-market-predictor\backend\venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: inv

,open,high,low,close,adjclose,volume,RSIadjclose15,RSIvolume15,RSIadjclose25,RSIvolume25,...,high-15,K-15,D-15,stochastic-k-15,stochastic-d-15,stochastic-kd-15,volumenrelativo,diff,INCREMENTO,TARGET
count,7781.000000,7781.000000,7781.000000,7781.000000,7781.000000,7.781000e+03,7316.000000,7316.000000,7006.000000,7006.000000,...,7347.000000,7262.000000,7194.000000,7262.000000,7194.000000,7194.000000,7566.000000,7626.000000,7626.000000,7781.000000
mean,34.990220,35.655999,34.301243,34.964414,34.483147,7.586022e+05,46.817434,49.814790,46.966016,49.898659,...,37.947291,18.673824,18.704812,18.673824,18.704812,0.298413,inf,-0.259186,-2.674224,0.183010
std,99.841502,101.451058,98.073945,99.790823,98.603879,3.934491e+06,11.672838,5.002664,8.760961,3.420371,...,107.340294,75.723295,74.210933,75.723295,74.210933,14.661948,NaN,7.334250,268.268134,0.386699
min,0.410000,0.435000,0.405000,0.408000,0.408000,0.000000e+00,6.837461,35.303213,17.693637,39.520876,...,0.510000,-668.212635,-626.263336,-668.212635,-626.263336,-211.219037,0.000000,-90.538818,-23399.465955,0.000000
25%,4.050000,4.130000,3.980000,4.030000,3.960000,1.080000e+04,38.946316,47.182234,40.954487,48.266978,...,4.565000,6.153839,8.336837,6.153839,8.336837,-6.585432,0.637237,-0.417873,-4.494383,0.000000
50%,10.080000,10.110000,10.005000,10.080000,10.061000,8.406000e+04,46.259711,48.356834,46.459477,48.961162,...,10.640000,28.484828,28.478797,28.484828,28.478797,0.000000,1.000000,-0.025000,-0.304004,0.000000
75%,24.350000,24.500000,24.080000,24.250000,22.466007,6.724000e+05,54.061089,50.902284,52.289893,50.527067,...,25.170000,59.688404,58.664021,59.688404,58.664021,6.726947,1.655385,0.240000,2.812552,0.000000
max,795.739990,799.359985,784.960022,797.489990,783.376221,1.615550e+08,96.365095,99.622735,91.023108,97.782293,...,799.359985,100.000000,100.000000,100.000000,100.000000,198.156313,inf,120.256775,425.000000,1.000000


In [15]:
df_stock.isnull().sum()


date                  0
open                  0
high                  0
low                   0
close                 0
                   ... 
stochastic-kd-15    587
volumenrelativo     215
diff                155
INCREMENTO          155
TARGET                0
Length: 1285, dtype: int64

In [16]:
new_df = df_stock.dropna()
new_df

,date,open,high,low,close,adjclose,volume,ticker,RSIadjclose15,RSIvolume15,...,high-15,K-15,D-15,stochastic-k-15,stochastic-d-15,stochastic-kd-15,volumenrelativo,diff,INCREMENTO,TARGET
98,2022-05-24,13.810000,14.120000,13.610000,14.100000,14.100000,74700,ASLE,47.575179,48.098729,...,15.09,57.142875,47.186159,57.142875,47.186159,9.956716,0.644521,-0.060000,-0.423732,0
99,2022-05-25,14.100000,14.320000,14.050000,14.180000,14.180000,71400,ASLE,48.459097,47.832464,...,14.76,70.707082,59.066871,70.707082,59.066871,11.640211,0.616048,0.210000,1.503221,0
100,2022-05-26,14.120000,14.347000,13.790000,13.820000,13.820000,89700,ASLE,44.815889,49.493682,...,14.76,52.525237,60.125065,52.525237,60.125065,-7.599827,0.773943,0.510000,3.831707,0
101,2022-05-27,14.010000,14.250000,13.940000,14.110000,14.110000,84800,ASLE,48.178500,49.045623,...,14.76,67.171696,63.468005,67.171696,63.468005,3.703691,0.731665,0.060000,0.427049,0
102,2022-05-31,14.140000,14.140000,13.530000,13.860000,13.860000,315400,ASLE,45.611408,65.015137,...,14.76,52.127632,57.274855,52.127632,57.274855,-5.147223,2.721311,0.750000,5.720824,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7771,2022-12-16,22.700001,22.879999,22.500000,22.670000,22.095634,13800,ATLO,45.827378,48.222673,...,23.60,-26.417248,-17.134822,-26.417248,-17.134822,-9.282426,0.938776,0.604294,2.811801,0
7772,2022-12-19,22.600000,22.799999,22.379999,22.600000,22.027409,144600,ATLO,44.016221,69.143430,...,23.60,-28.900837,-23.695948,-28.900837,-23.695948,-5.204889,9.836735,0.730997,3.432489,0
7773,2022-12-20,22.600000,23.000000,22.600000,22.840000,22.261328,21000,ATLO,51.113614,49.069797,...,23.60,-9.727156,-21.681747,-9.727156,-21.681747,11.954591,1.428571,0.497078,2.283919,0
7774,2022-12-21,22.889999,23.389999,22.889999,23.309999,22.719419,9600,ATLO,61.385254,47.701266,...,23.60,27.821310,-3.602228,27.821310,-3.602228,31.423537,0.653061,0.292400,1.303786,0


In [17]:
#compact pipeline: detects the price column, creates technical features (MA, returns, RSI), labels each day as `buy`/`sell`/`wait` using a configurable horizon and threshold, trains a `RandomForestClassifier`, evaluates it, and saves the model

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import numpy as np
from sklearn.utils import class_weight

#set parameters
HORIZON = 5          
THRESHOLD = 0.02     # 2% move
TEST_SIZE = 0.2      

_work = df_stock.copy()

close_candidates = [c for c in _work.columns if c.lower() in ['close','adj close','adj_close','adjusted_close','close_price']]
if close_candidates:
    CLOSE = close_candidates[0]
else:
    num_cols = _work.select_dtypes(include=np.number).columns.tolist()
    if not num_cols:
        raise ValueError('No numeric columns found to use as price')
    CLOSE = num_cols[-1]
print('Using close column: ', CLOSE)
_work['close'] = _work[CLOSE]

#tiwme ordering
date_cols = [c for c in _work.columns if 'date' in c.lower()]
if date_cols:
    _work['Date'] = pd.to_datetime(_work[date_cols[0]])
    _work = _work.sort_values('Date').reset_index(drop=True)
else:
    _work = _work.reset_index().rename(columns={'index':'DateIndex'})

for w in [1,3,5,10,20,50]:
    _work[f'ret_{w}'] = _work['close'].pct_change(w)
    _work[f'ma_{w}'] = _work['close'].rolling(w).mean()
_work['vol_10'] = _work['close'].pct_change().rolling(10).std()

def RSI(series, period=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -1*delta.clip(upper=0)
    ma_up = up.rolling(period).mean()
    ma_down = down.rolling(period).mean()
    rs = ma_up/ma_down
    return 100 - (100/(1+rs))

_work['rsi_14'] = RSI(_work['close'], 14)

#labels
_work['future_close'] = _work['close'].shift(-HORIZON)
_work['future_return'] = (_work['future_close'] - _work['close']) / _work['close']

def label_fn(x):
    if x > THRESHOLD:
        return 'buy'
    elif x < -THRESHOLD:
        return 'sell'
    else:
        return 'wait'

_work['action'] = _work['future_return'].apply(lambda x: label_fn(x) if pd.notnull(x) else np.nan)
_work = _work.dropna(subset=['action']).reset_index(drop=True)

print('Label distribution:')
print(_work['action'].value_counts())


Using close column:  close
Label distribution:
action
sell    3848
buy     3831
wait      97
Name: count, dtype: int64


In [19]:

#features and target
features = [c for c in _work.columns if c.startswith(('ret_','ma_','vol_','rsi_'))]
X = _work[features]
y = _work['action']

#train/test split
split_idx = int(len(_work)*(1-TEST_SIZE))
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
print('Train size:', len(X_train), 'Test size:', len(X_test))

#for class imbalance
classes = np.unique(y_train)
class_weights = 'balanced'
clf = RandomForestClassifier(n_estimators=200, random_state=random_seed, class_weight=class_weights)
clf.fit(X_train, y_train)

# Evaluation
y_pred = clf.predict(X_test)
print('\nClassification report:')
print(classification_report(y_test, y_pred))
print('\nConfusion matrix:')
print(confusion_matrix(y_test, y_pred))

# Feature importances
importances = pd.Series(clf.feature_importances_, index=features).sort_values(ascending=False)
print('\nTop features:')
print(importances.head(10))

import os
import joblib

# Anchors to the folder this notebook is in
notebook_dir = os.getcwd()
# Goes up to 'stock-market-predictor'
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
# Defines: stock-market-predictor/backend/model
target_dir = os.path.join(project_root, 'models')

os.makedirs(target_dir, exist_ok=True)
save_path = os.path.join(target_dir, 'stock_model.pkl')
joblib.dump(clf, save_path)

print(f"\n✅ SUCCESS! Model saved for the API at: {save_path}")

# Quick look at last predictions
window = 30
_recent = _work.iloc[-window:].copy()
_recent['pred'] = clf.predict(_recent[features])
cols_show = []
if 'Date' in _recent.columns:
    cols_show.append('Date')
cols_show += ['close','future_return','action','pred']
_recent[cols_show].head(10)

Train size: 6220 Test size: 1556

Classification report:
              precision    recall  f1-score   support

         buy       0.73      0.75      0.74       779
        sell       0.73      0.72      0.73       758
        wait       0.00      0.00      0.00        19

    accuracy                           0.73      1556
   macro avg       0.49      0.49      0.49      1556
weighted avg       0.72      0.73      0.73      1556


Confusion matrix:
[[588 191   0]
 [210 548   0]
 [  9  10   0]]

Top features:
ma_1      0.139866
ret_50    0.075115
ret_5     0.074276
ret_3     0.073843
ret_10    0.073286
rsi_14    0.070943
ret_1     0.068275
ret_20    0.064723
ma_3      0.062194
ma_20     0.061537
dtype: float64


c:\Users\HomePC\Documents\PROJECTS\stock-market-predictor\backend\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\HomePC\Documents\PROJECTS\stock-market-predictor\backend\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\HomePC\Documents\PROJECTS\stock-market-predictor\backend\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` par


✅ SUCCESS! Model saved for the API at: c:\Users\HomePC\Documents\PROJECTS\stock-market-predictor\backend\models\stock_model.pkl


,Date,close,future_return,action,pred
7746,2022-12-29,551.469971,-0.996736,sell,sell
7747,2022-12-29,52.410000,-0.933219,sell,sell
7748,2022-12-29,32.220001,-0.803228,sell,sell
7749,2022-12-29,1.925000,8.085714,buy,buy
7750,2022-12-30,21.309999,-0.238855,sell,sell
7751,2022-12-30,1.800000,-0.533333,sell,buy
7752,2022-12-30,3.500000,33.894285,buy,buy
7753,2022-12-30,6.340000,85.182967,buy,buy
7754,2022-12-30,17.490000,-0.465981,sell,sell
7755,2022-12-30,16.219999,-0.952528,sell,sell


In [20]:
date_cols = [c for c in _work.columns if 'date' in c.lower()]
if date_cols:
    _work['Date'] = pd.to_datetime(_work[date_cols[0]])
    _work = _work.sort_values('Date').reset_index(drop=True)
else:
    _work = _work.reset_index().rename(columns={'index':'DateIndex'})

# Features: returns, moving averages, vol, RSI
for w in [1,3,5,10,20,50]:
    _work[f'ret_{w}'] = _work['close'].pct_change(w)
    _work[f'ma_{w}'] = _work['close'].rolling(w).mean()
_work['vol_10'] = _work['close'].pct_change().rolling(10).std()

def RSI(series, period=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -1*delta.clip(upper=0)
    ma_up = up.rolling(period).mean()
    ma_down = down.rolling(period).mean()
    rs = ma_up/ma_down
    return 100 - (100/(1+rs))

_work['rsi_14'] = RSI(_work['close'], 14)
_work['future_close'] = _work['close'].shift(-HORIZON)
_work['future_return'] = (_work['future_close'] - _work['close']) / _work['close']

def label_fn(x):
    if x > THRESHOLD:
        return 'buy'
    elif x < -THRESHOLD:
        return 'sell'
    else:
        return 'wait'

_work['action'] = _work['future_return'].apply(lambda x: label_fn(x) if pd.notnull(x) else np.nan)
_work = _work.dropna(subset=['action']).reset_index(drop=True)

# print('Label distribution:')
# print(_work['action'].value_counts())